In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms
import os
import pandas as pd
from tqdm import tqdm
from transformers import AutoModel, AutoImageProcessor, AutoModelForImageClassification
import torch.nn.functional as F
from models improt MultimodalModel



# Define the dataset class
class MemeDataset(Dataset):
    def __init__(self, csv_file, image_folder, tokenizer, max_length, transform):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.transform = transform
        
        self.data = self.data[self.data['image_id'].apply(lambda x: os.path.isfile(os.path.join(self.image_folder, f"{x}.jpg")))]

        # Check if the image file exists for each row, raise an error if any file is missing
        missing_files = self.data[~self.data['image_id'].apply(lambda x: os.path.isfile(os.path.join(self.image_folder, f"{x}.jpg")))]

        if not missing_files.empty:
            raise FileNotFoundError(f"The following image files are missing: {', '.join(missing_files['image_id'].tolist())}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_id = row['image_id']
        label = row['labels']
        transcription = row['transcriptions'].lower()
        
        # Tokenize text
        text_inputs = self.tokenizer(
            transcription,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Load and transform image
        image_path = os.path.join(self.image_folder, f"{image_id}.jpg")
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)

        return {
            'text_inputs': {key: val.squeeze(0) for key, val in text_inputs.items()},
            'image': image,
            'label': torch.tensor(label, dtype=torch.long)
        }




# Training and evaluation setup
def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(dataloader, leave=True, desc="Training"):
        optimizer.zero_grad()
        text_inputs = {key: val.to(device) for key, val in batch['text_inputs'].items()}
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        loss, outputs = model(text_inputs, images, labels=labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    accuracy = correct / total
    return total_loss / len(dataloader), accuracy

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total = 0
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader, leave=True, desc="Evaluating"):
            text_inputs = {key: val.to(device) for key, val in batch['text_inputs'].items()}
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            loss, outputs = model(text_inputs, images, labels=labels)
            
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            predictions.extend(preds)
            true_labels.extend(labels)

    return total_loss / len(dataloader), predictions, true_labels



image_folder = "./malayalam/all"  # Path to the folder containing images
text_model_name = "bytesizedllm/MalayalamXLM_Roberta"
num_classes = 2
max_length = 64
batch_size = 16
num_epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer and transformations
tokenizer = AutoTokenizer.from_pretrained(text_model_name, cache_dir="./xlm_robertaMalayalam")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and DataLoader
train_dataset = MemeDataset("./malayalam/all/train.csv", image_folder, tokenizer, max_length, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = MemeDataset("./malayalam/all/dev.csv", image_folder, tokenizer, max_length, transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = MemeDataset("./malayalam/test_with_labels/test_with_labels.csv", image_folder, tokenizer, max_length, transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model, optimizer, and loss function
model = MultimodalModel(text_model_name, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2.5e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

from sklearn.metrics import accuracy_score, classification_report
# # Main Training Loop
best_macro_f1 = 0.0
best_model_path = "./malayalam/best_modelM.pth"

# Training loop
for epoch in range(num_epochs):
    print("Epoch: ", epoch)
    train_loss, train_acc = train_model(model, train_loader, optimizer, criterion, device)
    val_loss, val_predictions, val_true_labels = evaluate_model(model, val_loader, criterion, device)
    test_loss, test_predictions, test_true_labels = evaluate_model(model, test_loader, criterion, device)

    report = classification_report(test_true_labels, test_predictions)
    report1 = classification_report(test_true_labels, test_predictions, output_dict=True)
    macro_f1 = report1['macro avg']['f1-score']
    


    # Save best model
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        print(f"Test Macro F1-Score: {macro_f1:.4f}")
        print("Classification Report on Test:\n", report)
        print("Classification Report on Validation:\n", classification_report(val_true_labels, val_predictions))
        torch.save(model.state_dict(), best_model_path)
        print(f"New best Macro F1-Score: {best_macro_f1:.4f}. Saving model...")

print(f"Best Macro F1-Score achieved on Test set: {best_macro_f1:.4f}")




Some weights of XLMRobertaModel were not initialized from the model checkpoint at bytesizedllm/MalayalamXLM_Roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/rohit/anaconda3/envs/xlstm/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rohit/anaconda3/envs/xlstm/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch:  0


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  2.03it/s]


Test Macro F1-Score: 0.8598
Classification Report on Test:
               precision    recall  f1-score   support

           0       0.86      0.93      0.90       122
           1       0.88      0.77      0.82        78

    accuracy                           0.87       200
   macro avg       0.87      0.85      0.86       200
weighted avg       0.87      0.87      0.87       200

Classification Report on Validation:
               precision    recall  f1-score   support

           0       0.83      0.94      0.88        97
           1       0.88      0.71      0.79        63

    accuracy                           0.85       160
   macro avg       0.86      0.83      0.84       160
weighted avg       0.85      0.85      0.85       160

New best Macro F1-Score: 0.8598. Saving model...
Epoch:  1


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  2.10it/s]


Test Macro F1-Score: 0.8648
Classification Report on Test:
               precision    recall  f1-score   support

           0       0.86      0.94      0.90       122
           1       0.90      0.77      0.83        78

    accuracy                           0.88       200
   macro avg       0.88      0.86      0.86       200
weighted avg       0.88      0.88      0.87       200

Classification Report on Validation:
               precision    recall  f1-score   support

           0       0.87      0.96      0.91        97
           1       0.92      0.78      0.84        63

    accuracy                           0.89       160
   macro avg       0.90      0.87      0.88       160
weighted avg       0.89      0.89      0.89       160

New best Macro F1-Score: 0.8648. Saving model...
Epoch:  2


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:05<00:00,  2.21it/s]


Test Macro F1-Score: 0.8805
Classification Report on Test:
               precision    recall  f1-score   support

           0       0.92      0.89      0.90       122
           1       0.83      0.88      0.86        78

    accuracy                           0.89       200
   macro avg       0.88      0.88      0.88       200
weighted avg       0.89      0.89      0.89       200

Classification Report on Validation:
               precision    recall  f1-score   support

           0       0.87      0.96      0.91        97
           1       0.92      0.78      0.84        63

    accuracy                           0.89       160
   macro avg       0.90      0.87      0.88       160
weighted avg       0.89      0.89      0.89       160

New best Macro F1-Score: 0.8805. Saving model...
Epoch:  3


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  1.86it/s]
